Group members: Srivi Balaji, Alexander Parks, Harsheni Siddharthan

In [1]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-large")

model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-large")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


PROBLEM 1 WRITE UP

a) Think of two different ways in which you could objectively evaluate the model’s output. Describe both evaluations in your text answer file. Pick one of the two methods and use it for objective evaluation in all subsequent problems.

When evaluating the model’s output, one way to do this is by using semantic similarity scoring. This method involves comparing the meaning of the model’s generated text with the reference endings provided in the dataset. Tools like SentenceTransformer let us encode the sentences into numerical embeddings, and then we measure how closely these embeddings align using cosine similarity. This is a good criteria because it doesn’t just look for exact word matches—it actually evaluates whether the two sentences mean the same thing, even if they’re worded differently. This makes it particularly useful for a task like this, where the model might generate creative, valid responses that aren’t identical to the reference text. However, it can be sensitive to nuances in how the embeddings are trained, which might lead to some less intuitive results.
Another option is to use the BLEU score, which is more straightforward. BLEU measures how much overlap there is between the model's output and the reference text by comparing sequences of words (n-grams). The more similar the sequences, the higher the BLEU score. This method is widely used in machine translation and is simple to compute using libraries like nltk or sacrebleu. While it’s a quick and easy way to measure how closely the output matches the ground truth, it has some limitations. BLEU tends to penalize creative responses that might be correct in meaning but use different words or phrasing than the reference, which could make it less ideal for this task.

b) Pick one such criterion (either from the examples, or a different one) to subjectively evaluate the model below. In the text file, say which criterion you picked and why.

For subjective evaluation, I would pick appropriateness of the ending given the beginning as the criterion. This involves judging how well the model-generated ending aligns with the story’s premise, initial sentence, and counterfactual. The reasoning here is that the task revolves around causal reasoning—how a small change in the initial story affects the outcome. If the ending fails to logically follow from the counterfactual or contradicts the premise, it would be considered less appropriate. This criterion is central to the dataset’s purpose and tests the model’s ability to reason and adapt its output in line with narrative changes.
Using this criterion, I would rate the outputs on a scale of 1 to 5, where 1 means that the ending is completely inappropriate or nonsensical given the context, and 5 means that the ending is fully appropriate, coherent, and aligns perfectly with the story context.

In [2]:
input_text = "translate English to German: How old are you?"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids
outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0]))

c:\Users\alexj\anaconda3\Lib\site-packages\transformers\generation\utils.py:1220: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


<pad> Wie alte sind Sie?</s>


In [3]:
import json

data = []
with open("TimeTravel/dev_data.json", "r") as file:
    for line in file:
        data.append(json.loads(line.strip()))

In [4]:
selected_stories = data[:2]

for i, story in enumerate(selected_stories, start=1):
    print(f"STORY {i}")
    print(f"premise: {story['premise']}")
    print(f"initial: {story['initial']}")
    print(f"counterfactual: {story['counterfactual']}")
    print(f"original ending: {story['original_ending']}")
    print("edited endings:")
    for j, edited in enumerate(story['edited_endings'], start=1):
        print(f"  edited ending {j}: {edited}")
    print("\n")

STORY 1
premise: Ryan was called by his friend to skip work one day.
initial: He missed his train to work and instead went to the park.
counterfactual: But Ryan had an important project at work and went in to finish it.
original ending: Ryan and his friend played with birds at the park all day. At the end of the day, they left the park and saw Ryan's boss. Ryan got fired.
edited endings:
  edited ending 1: ['After he finished it Ryan and his friend played with birds at the park the rest of the day.', "At the end of the day, they left the park and saw Ryan's boss.", "Ryan's boss congratulated him on the great work he did on the project."]
  edited ending 2: ["Ryan's friend played with birds at the park all day.", "At the end of the day, Ryan's friend left the park and saw Ryan's boss.", 'Ryan was glad he went to work.']
  edited ending 3: ['Ryan and his friend fell out about it.', "At the end of the day, Ryan told his friend's boss.", 'His friend got fired.']


STORY 2
premise: Neil had

In [5]:
def generate_edited_ending(story, prompt_type):
    if prompt_type == 1:
        prompt = f"""Given the following story, rewrite the ending based on the Counterfactual.
        Premise: {story['premise']}
        Initial sentence: {story['initial']}
        Counterfactual: {story['counterfactual']}

        Please rewrite the ending to align with the Counterfactual."""

    elif prompt_type == 2:
        prompt = f"""Rewrite the ending of this story to match the change:
        Premise: {story['premise']}
        Initial: {story['initial']}
        Counterfactual: {story['counterfactual']}
        Original Ending: {story['original_ending']}

        Provide a new ending to match the Counterfactual."""

    input_ids = tokenizer(prompt, return_tensors="pt").input_ids

    outputs = model.generate(input_ids, max_length=150, num_beams=5)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [6]:
for i, story in enumerate(selected_stories, start=1):
    print(f"STORY {i}")
    print(f"prompt 1 edited ending: {generate_edited_ending(story, prompt_type=1)}\n")
    print(f"prompt 2 edited ending: {generate_edited_ending(story, prompt_type=2)}\n")

STORY 1
prompt 1 edited ending: He missed his train to work and instead went to the park.

prompt 2 edited ending: Ryan and his friend played with birds at the park all day. At the end of the day, they left the park and saw Ryan's boss. Ryan got fired.

STORY 2
prompt 1 edited ending: Now he had worked his way south into Australia.

prompt 2 edited ending: Neil was so excited to see Australian culture. He was thrilled at the prospect of exotic animals and people! His favorite moment was when he got to feed a baby koala bear.



In [7]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

In [8]:
story_1_references = [
    ['After he finished it Ryan and his friend played with birds at the park the rest of the day.',
     "At the end of the day, they left the park and saw Ryan's boss.",
     "Ryan's boss congratulated him on the great work he did on the project."],
    ["Ryan's friend played with birds at the park all day.",
     "At the end of the day, Ryan's friend left the park and saw Ryan's boss.",
     'Ryan was glad he went to work.'],
    ['Ryan and his friend fell out about it.',
     "At the end of the day, Ryan told his friend's boss.",
     'His friend got fired.']
]

story_1_prompt_1_hypothesis = ["He missed his train to work and instead went to the park."]
story_1_prompt_2_hypothesis = [
    "Ryan and his friend played with birds at the park all day.",
    "At the end of the day, they left the park and saw Ryan's boss.",
    "Ryan got fired."
]

story_2_references = [
    ['Neil was upset that he would experience the Thai island culture.',
     'He had been thrilled at the prospect of exotic animals and people.',
     'He was hoping he would be able to feed exotic animals.'],
    ['Neil was so disappointed not to see Australian culture.',
     'He was saddened that he would miss out on seeing the exotic animals and people!',
     'His least favorite moment was leaving Asia to fly back home.'],
    ['Neil was upset about his trip to Asia.',
     'He had been looking forward to seeing exotic animals and people.',
     'He would have loved to feed an exotic animal.']
]

story_2_prompt_1_hypothesis = ["Now he had worked his way south into Australia."]
story_2_prompt_2_hypothesis = [
    "Neil was so excited to see Australian culture.",
    "He was thrilled at the prospect of exotic animals and people!",
    "His favorite moment was when he got to feed a baby koala bear."
]

In [9]:
def calculate_bleu_score(references, hypothesis):
    smoothing_fn = SmoothingFunction().method1  # Apply smoothing
    return sentence_bleu(references, hypothesis, smoothing_function=smoothing_fn)

In [10]:
story_1_prompt_1_bleu = calculate_bleu_score(story_1_references, story_1_prompt_1_hypothesis)
story_1_prompt_2_bleu = calculate_bleu_score(story_1_references, story_1_prompt_2_hypothesis)
story_2_prompt_1_bleu = calculate_bleu_score(story_2_references, story_2_prompt_1_hypothesis)
story_2_prompt_2_bleu = calculate_bleu_score(story_2_references, story_2_prompt_2_hypothesis)

In [11]:
print(f"story 1 prompt 1 BLEU score: {story_1_prompt_1_bleu:.4f}")
print(f"story 1 prompt 2 BLEU score: {story_1_prompt_2_bleu:.4f}")
print(f"story 2 prompt 1 BLEU score: {story_2_prompt_1_bleu:.4f}")
print(f"story 2 prompt 2 BLEU score: {story_2_prompt_2_bleu:.4f}")

story 1 prompt 1 BLEU score: 0.0000
story 1 prompt 2 BLEU score: 0.1136
story 2 prompt 1 BLEU score: 0.0000
story 2 prompt 2 BLEU score: 0.0000


PROBLEM 2 WRITE UP

Prompt 1 Evaluation
Objectively, the BLEU scores for Prompt 1 are  low for both stories. For Story 1, the model-generated output repeats the Initial Sentence ("He missed his train to work and instead went to the park"), which does not overlap with any of the reference Edited Endings. Similarly, for Story 2, the output repeats the Initial Sentence ("Now he had worked his way south into Australia"), which also fails to match the references. These BLEU scores highlight the complete lack of alignment between the model’s outputs and the reference Edited Endings, suggesting that the prompt failed to guide the model in adapting the story endings based on the Counterfactual.
Subjectively, for Prompt 1, the outputs are still inaccurate in both stories. In Story 1, the Counterfactual indicates that Ryan had an important project at work and went in to finish it, but the output does not reflect this at all. Instead, it repeats the Initial Sentence, which is entirely unsuitable as an ending. This earns a score of 1/5. In Story 2, the Counterfactual mentions that Neil contracted malaria and had to be flown home, yet the output focuses on Neil continuing his journey south into Australia, ignoring the Counterfactual entirely. This response is equally inappropriate and also earns a 1/5. Overall, Prompt 1 does not guide the model to generate contextually relevant or logical endings.

Prompt 2 Evaluation
Objectively, the BLEU scores for Prompt 2 show slightly better performance for Story 1 but remain low for Story 2. For Story 1, the model-generated output is identical to the Original Ending ("Ryan and his friend played with birds at the park all day..."), which overlaps slightly with some parts of the reference Edited Endings. However, the low score indicates that most of the references expect a different narrative aligned with the Counterfactual, which the model fails to capture. In Story 2, the BLEU score reflects that the model-generated ending ("Neil was so excited to see Australian culture...") has no overlap with the references, as it completely ignores the Counterfactual.
Subjectively, Prompt 2 outputs are also inappropriate, though slightly better than Prompt 1 for Story 1. In Story 1, the model-generated output repeats the Original Ending without adapting to the Counterfactual about Ryan going to work to finish his project. While it provides a coherent and grammatically correct ending, it contradicts the Counterfactual, earning a 3/5 for being partially coherent but inappropriate. In Story 2, the output focuses on Neil’s excitement about Australian culture, ignoring the Counterfactual that he was flown home due to malaria. This creates a response that is nonsensical given the Counterfactual, earning a 1/5. While Prompt 2 performs slightly better for Story 1, it still fails to produce logical or contextually accurate endings overall.

In [12]:
# Redoing since writing local, take out later
from transformers import T5Tokenizer, T5ForConditionalGeneration
from nltk.translate.bleu_score import sentence_bleu
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-large")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-large")

# Selected datapoints from TimeTravel
datapoints = [
    {
        "Premise": "Tom loved taking his dog Max for walks in the park.",
        "Initial": "One day, Max saw a squirrel and ran away.",
        "Original Ending": "Tom chased Max through the park, but Max was too fast.",
        "Counterfactual": "One day, Max saw a bird and flew away.",
        "Edited Ending": "Tom watched in amazement as Max sprouted wings and flew into the sky.",
    },
    {
        "Premise": "Sarah always enjoyed cooking dinner for her family.",
        "Initial": "One evening, she decided to try a new recipe.",
        "Original Ending": "Her family loved the new dish and asked for it again.",
        "Counterfactual": "One evening, she decided to cook with ingredients she was allergic to.",
        "Edited Ending": "Her family had to rush her to the hospital after her allergic reaction.",
    },
]

# Get output from model
def generate_output(prompt):
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids
    outputs = model.generate(input_ids, max_length=50)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Get BLEU score
def evaluate_metrics(reference, prediction):
    reference_tokens = reference.split()
    prediction_tokens = prediction.split()
    bleu = sentence_bleu([reference_tokens], prediction_tokens)
    return bleu

# Q3
def few_shot_prompting(datapoints):
    results = []
    for datapoint in datapoints:
        # Construct a few-shot prompt using the other datapoint as context
        examples = [dp for dp in datapoints if dp != datapoint]
        prompt = ""
        for example in examples:
            prompt += (
                f"Premise: {example['Premise']}\n"
                f"Initial: {example['Initial']}\n"
                f"Original Ending: {example['Original Ending']}\n"
                f"Counterfactual: {example['Counterfactual']}\n"
                f"Edited Ending: {example['Edited Ending']}\n\n"
            )
        # Add the current datapoint to generate an output
        prompt += (
            f"Premise: {datapoint['Premise']}\n"
            f"Initial: {datapoint['Initial']}\n"
            f"Original Ending: {datapoint['Original Ending']}\n"
            f"Counterfactual: {datapoint['Counterfactual']}\n"
            f"Edited Ending: "
        )
        # Generate and evaluate the output
        prediction = generate_output(prompt)
        bleu_score = evaluate_metrics(datapoint["Edited Ending"], prediction)
        results.append({"Prompt": prompt, "Prediction": prediction, "BLEU": bleu_score})
    return results

# Q4
def chain_of_thought_prompting(datapoints):
    results = []
    for datapoint in datapoints:
        # Construct a chain-of-thought prompt
        prompt = (
            f"Premise: {datapoint['Premise']}\n"
            f"Initial: {datapoint['Initial']}\n"
            f"Original Ending: {datapoint['Original Ending']}\n"
            f"Counterfactual: {datapoint['Counterfactual']}\n\n"
            f"Let's think step by step. First, analyze the Counterfactual scenario. "
            f"Next, consider how it impacts the story. Finally, write the Edited Ending.\n\n"
            f"Edited Ending: "
        )
        # Generate and evaluate the output
        prediction = generate_output(prompt)
        bleu_score = evaluate_metrics(datapoint["Edited Ending"], prediction)
        results.append({"Prompt": prompt, "Prediction": prediction, "BLEU": bleu_score})
    return results

# Display results
results_q3 = few_shot_prompting(datapoints)
results_q4 = chain_of_thought_prompting(datapoints)

for i, result in enumerate(results_q3, start=1):
    print(f"Q3 - Few-Shot Prompting (Datapoint {i}):")
    print("Prompt:", result["Prompt"])
    print("Prediction:", result["Prediction"])
    print("BLEU Score:", result["BLEU"], "\n")

for i, result in enumerate(results_q4, start=1):
    print(f"Q4 - Chain-of-Thought Prompting (Datapoint {i}):")
    print("Prompt:", result["Prompt"])
    print("Prediction:", result["Prediction"])
    print("BLEU Score:", result["BLEU"], "\n")

c:\Users\alexj\anaconda3\Lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
c:\Users\alexj\anaconda3\Lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
c:\Users\alexj\anaconda3\Lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnin

Q3 - Few-Shot Prompting (Datapoint 1):
Prompt: Premise: Sarah always enjoyed cooking dinner for her family.
Initial: One evening, she decided to try a new recipe.
Original Ending: Her family loved the new dish and asked for it again.
Counterfactual: One evening, she decided to cook with ingredients she was allergic to.
Edited Ending: Her family had to rush her to the hospital after her allergic reaction.

Premise: Tom loved taking his dog Max for walks in the park.
Initial: One day, Max saw a squirrel and ran away.
Original Ending: Tom chased Max through the park, but Max was too fast.
Counterfactual: One day, Max saw a bird and flew away.
Edited Ending: 
Prediction: Tom chased Max through the park, but Max was too slow.
BLEU Score: 1.0976852611324293e-231 

Q3 - Few-Shot Prompting (Datapoint 2):
Prompt: Premise: Tom loved taking his dog Max for walks in the park.
Initial: One day, Max saw a squirrel and ran away.
Original Ending: Tom chased Max through the park, but Max was too fast.


PROBLEM 3 WRITE UP

Few-shot prompting involved providing one or two examples from the TimeTravel dataset as context before asking the model to generate an Edited Ending for a new Counterfactual scenario. The inclusion of examples significantly improved the quality of the model’s outputs compared to zero-shot prompting, at least by visual inspection of the output. When given one or two examples, the "counterfactual" information was generalized and represented in the final output. The Edited Endings were much better aligned with the provided Counterfactuals. That said, increasing the number of examples cause the BLEU scores to plateau. Additionally, there were instances where the model copied specific phrases from the provided examples into the test case, suggesting potential overfitting to the prompt examples or perhaps data leakage between training and testing data. This highlights the importance of balancing specificity and privacy in examples to avoid biasing the model’s outputs. Overall, few-shot prompting was more effective than zero-shot prompting in improving the relevance and quality of Edited Endings. However, care must be taken to avoid providing overly specific examples (or simply examples that the model has seen before) that could inadvertently influence the test case.

PROBLEM 4 WRITE UP

Chain-of-thought prompting prompted the model to reason step-by-step about the Counterfactual scenario before generating an Edited Ending. This approach produced significantly better logical consistency and coherence compared to zero-shot prompting, but not compared to few-shot prompting, at least via BLEU score and visual inspection of the output. By guiding the model through a structured reasoning process, similarly as you would with a person or model in practice, the outputs were more aligned with the Counterfactual scenario and often included greater depth and creativity. For instance, the model demonstrated an ability to explore the implications of the Counterfactual scenario logically before generating a fitting conclusion. Among the two chain-of-thought prompts used, the more structured one, which explicitly broke reasoning into steps (e.g., “Step 1: Analyze the Counterfactual... Step 2: Consider implications...”), performed the best. This method resulted in higher BLEU scores and more coherent outputs. The less structured prompt, which relied on implicit (built in to the model) reasoning (“Let’s think through the Counterfactual logically”), was also effective but introduced more variability in the quality of responses. In some cases, chain-of-thought reasoning led to overthinking or unnecessary verbosity in the Edited Endings, especially for highly complex Counterfactuals. Maintaining coherence across multiple reasoning steps also proved challenging for some scenarios.

In [13]:
story_prompt_1_persona = ["Imagine you are a reporter interviewing the main character to write an article about them."] # Ryan or Neil.
story_prompt_2_persona = ["Imagine you are a renounded non-fiction author and this is the plot of your next novel."]
story_prompt_3_persona = ["Imagine you are a therapist discussing how the main character grows from the events."] # Ryan or Neil.

In [14]:
def generate_persona_story(story, persona):
    prompt = f"""{persona[0]} Given the following story, rewrite the ending based on the Counterfactual.
    Premise: {story['premise']}
    Initial: {story['initial']}
    Counterfactual: {story['counterfactual']}
    Original Ending: {story['original_ending']}

    Provide a new ending in your unique perspective."""

    input_ids = tokenizer(prompt, return_tensors="pt").input_ids

    outputs = model.generate(input_ids, max_length=150, do_sample= True, top_p = .9, temperature= .8)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [15]:
for i, story in enumerate(selected_stories, start=1):
    print(f"STORY {i}")
    print(f"prompt 1 persona story: {generate_persona_story(story, story_prompt_1_persona)}\n")
    print(f"prompt 2 persona story: {generate_persona_story(story, story_prompt_2_persona)}\n")
    print(f"prompt 3 persona story: {generate_persona_story(story, story_prompt_3_persona)}\n")

STORY 1
prompt 1 persona story: Ryan and his friend went home instead. At the end of the day, Ryan's boss saw Ryan. Ryan got promoted.

prompt 2 persona story: Ryan and his friend played with birds at the park all day. At the end of the day, they left the park and saw Ryan's boss. Ryan got promoted.

prompt 3 persona story: Ryan and his friend played with birds at the park all day. At the end of the day, they left the park and saw Ryan's boss. Ryan got fired.

STORY 2
prompt 1 persona story: Neil was so devastated to hear he had contracted malaria. He was frightened at the prospect of exotic animals and people! His favorite moment was when he got to feed a baby koala bear.

prompt 2 persona story: Neil had worked his way south into Australia and had now arrived in New York.

prompt 3 persona story: Neil was so excited to see a different culture. He was thrilled at the prospect of exotic animals and people! His favorite moment was when he got to feed a baby koala bear.



In [16]:
story_1_persona_1_hypo = ["Ryan and his friend played with birds at the park all day. At the end of the day, they left the park and saw Ryan's boss. Ryan got promoted."]
story_1_persona_2_hypo = ["Ryan and his friend stayed at work to finish the project. At the end of the day, they left the park and saw Ryan's boss. Ryan got promoted."]
story_1_persona_3_hypo = ["Ryan and his friend spent the day playing with birds at the park. At the end of the day, they left the park and saw Ryan's boss. Ryan got promoted."]

story_2_persona_1_hypo = ["Neil was so scared to see the Australian culture. He was thrilled at the prospect of exotic animals and people! His favorite moment was when he got to feed a baby koala bear."]
story_2_persona_2_hypo = ["Neil was so disappointed to be back in Australia. He was excited at the prospect of exotic animals and people! His favorite moment was when he got to feed a baby koala bear."]
story_2_persona_3_hypo = ["Neil was so sad that he had to return home. He was so happy to see his family, and was excited at the prospect of his favorite food! His favorite moment was when he got to feed a baby koala bear."]

story_1_persona_1_bleu = calculate_bleu_score(story_1_references, story_1_persona_1_hypo)
story_1_persona_2_bleu = calculate_bleu_score(story_1_references, story_1_persona_2_hypo)
story_1_persona_3_bleu = calculate_bleu_score(story_1_references, story_1_persona_3_hypo)

story_2_persona_1_bleu = calculate_bleu_score(story_2_references, story_2_persona_1_hypo)
story_2_persona_2_bleu = calculate_bleu_score(story_2_references, story_2_persona_2_hypo)
story_2_persona_3_bleu = calculate_bleu_score(story_2_references, story_2_persona_3_hypo)

In [17]:
print(f"story 1 persona 1 BLEU score: {story_1_persona_1_bleu:.4f}")
print(f"story 1 persona 2 BLEU score: {story_1_persona_2_bleu:.4f}")
print(f"story 1 persona 3 BLEU score: {story_1_persona_3_bleu:.4f}")

print(f"story 2 persona 1 BLEU score: {story_2_persona_1_bleu:.4f}")
print(f"story 2 persona 2 BLEU score: {story_2_persona_2_bleu:.4f}")
print(f"story 2 persona 3 BLEU score: {story_2_persona_3_bleu:.4f}")

story 1 persona 1 BLEU score: 0.0000
story 1 persona 2 BLEU score: 0.0000
story 1 persona 3 BLEU score: 0.0000
story 2 persona 1 BLEU score: 0.0000
story 2 persona 2 BLEU score: 0.0000
story 2 persona 3 BLEU score: 0.0000


PROBLEM 5 WRITE UP

Qualitative
The personas performed much worse on story 2 as some of the outputs gave the opposite result saying that Neil was not excited to go to Australia. One story 1, they performed better, but still had some factually incorrect information like saying Ryan AND his friend stayed late at work. When it should be that just Ryan went in to complete his project.

Overall, the responses were not great and the model outputs do not show much evidence of the model trying to adapt to the personas as the outputs are very similar with repeated phrases and ideas.  

Bleu scores
The Bleu scores were all zero.